In [22]:
import time
import re
import pylab as pl
import numpy as np
import pylab as pl

from datetime import datetime, timedelta

import subprocess

import os
import pandas as pd

import concurrent.futures


from joblib import Parallel, delayed
import shutil


from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [23]:
gcp_username = 'tejas'

In [24]:
num_nodes = 3

In [25]:

commands  = []



for i in range(num_nodes):
    cmd =  'gcloud compute instances create nuraft-'+f"{i:03}"+' --project=ucr-ursa-major-lesani-lab \
    --zone=us-central1-c \
    --machine-type=e2-highcpu-4 \
    --network-interface=network-tier=PREMIUM,stack-type=IPV4_ONLY,subnet=default \
    --can-ip-forward \
    --maintenance-policy=MIGRATE \
    --provisioning-model=STANDARD \
    --service-account=961693926925-compute@developer.gserviceaccount.com \
    --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/trace.append \
    --tags=http-server,https-server \
    --create-disk=auto-delete=yes,boot=yes,device-name=instance-20250611-154850,disk-resource-policy=projects/ucr-ursa-major-lesani-lab/regions/us-central1/resourcePolicies/default-schedule-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-minimal-2504-plucky-amd64-v20250606,mode=rw,size=10,type=pd-balanced \
    --no-shielded-secure-boot \
    --shielded-vtpm \
    --shielded-integrity-monitoring \
    --labels=goog-ec-src=vm_add-gcloud \
    --reservation-affinity=any'

    commands.append(cmd)


def run_command(command):
    subprocess.call(command, shell=True)


# Create a ThreadPoolExecutor or ProcessPoolExecutor, depending on your needs
with concurrent.futures.ThreadPoolExecutor(max_workers=60) as executor:
    # Submit each command to the executor for parallel execution
    futures = [executor.submit(run_command, command) for command in commands]

    # Wait for all tasks to complete
    concurrent.futures.wait(futures)

Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-002].


NAME        ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
nuraft-002  us-central1-c  e2-highcpu-4               10.128.0.57  35.222.80.144  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-001].


NAME        ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP     STATUS
nuraft-001  us-central1-c  e2-highcpu-4               10.128.0.54  34.133.142.103  RUNNING


Created [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-000].


NAME        ZONE           MACHINE_TYPE  PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP   STATUS
nuraft-000  us-central1-c  e2-highcpu-4               10.128.0.53  34.41.232.84  RUNNING


In [26]:
redirect = '>'

os.system('gcloud compute instances list  --format="value(networkInterfaces[0].networkIP)" '+\
          redirect+' ../all_internal_ips')

0

In [27]:
f = open('../all_internal_ips', "r")
iplist = [line.strip() for line in f.readlines()]

In [28]:
iplist

['10.128.0.53', '10.128.0.54', '10.128.0.57']

In [29]:
def setup_remote(i):


    command = 'gcloud compute ssh --zone "'+'us-central1-c'+'" "nuraft-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "\
    sudo apt update; \
    sudo apt -y install build-essential git cmake openssl libssl-dev libz-dev pkg-config libzstd-dev;\
    git clone https://github.com/tejas-shivanand-mane/NuRaft.git"'

    print(command)

    output = os.system(command)
    print(output)


In [30]:
results = Parallel(n_jobs=60)(delayed(setup_remote)(i) for i in range(len(iplist)) )


print(results)  

Get:1 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky InRelease [265 kB]
Get:2 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-updates InRelease [126 kB]
Get:3 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-backports InRelease [126 kB]
Get:4 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/main amd64 Packages [1446 kB]
Get:5 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/main Translation-en [519 kB]
Get:6 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/universe amd64 Packages [16.3 MB]


Get:7 http://security.ubuntu.com/ubuntu plucky-security InRelease [126 kB]
Get:1 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky InRelease [265 kB]
Get:2 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-updates InRelease [126 kB]
Get:3 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-backports InRelease [126 kB]
Get:8 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/universe Translation-en [6281 kB]
Get:4 http://security.ubuntu.com/ubuntu plucky-security InRelease [126 kB]
Get:5 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/main amd64 Packages [1446 kB]
Get:9 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/restricted amd64 Packages [52.3 kB]
Get:10 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/restricted Translation-en [13.1 kB]
Get:11 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/multiverse amd64 Packages [260 kB]
Get:12 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/multiverse Translation-en [119 kB]
Get:13 h

Get:22 http://security.ubuntu.com/ubuntu plucky-security/main Translation-en [22.8 kB]
Get:23 http://security.ubuntu.com/ubuntu plucky-security/universe amd64 Packages [65.5 kB]
Get:24 http://security.ubuntu.com/ubuntu plucky-security/universe Translation-en [21.9 kB]
Get:25 http://security.ubuntu.com/ubuntu plucky-security/restricted amd64 Packages [46.6 kB]
Get:1 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky InRelease [265 kB]
Get:26 http://security.ubuntu.com/ubuntu plucky-security/restricted Translation-en [11.8 kB]
Get:27 http://security.ubuntu.com/ubuntu plucky-security/multiverse amd64 Packages [14.0 kB]
Get:28 http://security.ubuntu.com/ubuntu plucky-security/multiverse Translation-en [2836 B]
Get:2 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-updates InRelease [126 kB]
Get:3 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky-backports InRelease [126 kB]
Get:4 http://us-central1.gce.archive.ubuntu.com/ubuntu plucky/main amd64 Packages [1446 kB]
Get:5 h


Reading state information...
13 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...


openssl is already the newest version (3.4.1-1ubuntu3).
openssl set to manually installed.
Solving dependencies...
Reading state information...
Fetched 26.2 MB in 3s (9018 kB/s)
Reading package lists...
openssl is already the newest version (3.4.1-1ubuntu3).
openssl set to manually installed.
Solving dependencies...
Installing:
  build-essential  git         libzstd-dev  zlib1g-dev
  cmake            libssl-dev  pkg-config

Installing dependencies:
  binutils                   gpg-agent                  libisl23
  binutils-common            gpg-wks-client             libitm1
  binutils-x86-64-linux-gnu  gpgconf                    libjansson4
  bzip2                      gpgsm                      libjsoncpp26
  cmake-data                 keyboxd                    libksba8
  cpp                        less                       liblsan0
  cpp-14                     libalgorithm-diff-perl     libmpc3
  cpp-14-x86-64-linux-gnu    libalgorithm-diff-xs-perl  libmpfr6
  cpp-x86-64-linux-gnu

...

Selecting previously unselected package libpkgconf3:amd64.
Selecting previously unselected package g++-x86-64-linux-gnu.
Preparing to unpack .../73-libpkgconf3_1.8.1-4_amd64.deb ...
Unpacking libpkgconf3:amd64 (1.8.1-4) ...
Preparing to unpack .../35-g++-x86-64-linux-gnu_4%3a14.2.0-1ubuntu1_amd64.deb ...
Unpacking g++-x86-64-linux-gnu (4:14.2.0-1ubuntu1) ...
Selecting previously unselected package libssl-dev:amd64.
Preparing to unpack .../74-libssl-dev_3.4.1-1ubuntu3_amd64.deb ...
Unpacking libssl-dev:amd64 (3.4.1-1ubuntu3) ...
Selecting previously unselected package g++.
Preparing to unpack .../36-g++_4%3a14.2.0-1ubuntu1_amd64.deb ...
Unpacking g++ (4:14.2.0-1ubuntu1) ...
Selecting previously unselected package make.
Preparing to unpack .../37-make_4.4.1-1_amd64.deb ...
Unpacking make (4.4.1-1) ...
Selecting previously unselected package perl-modules-5.40.
Preparing to unpack .../38-perl-modules-5.40_5.40.1-2ubuntu0.1_all.deb ...
Unpacking perl-modules-5.40 (5.40.1-2ubuntu0.1) ...
Sel

Cloning into 'NuRaft'...


Unpacking gpg-agent (2.4.4-2ubuntu23) ...
Selecting previously unselected package gpgsm.
Preparing to unpack .../64-gpgsm_2.4.4-2ubuntu23_amd64.deb ...
Unpacking gpgsm (2.4.4-2ubuntu23) ...
Selecting previously unselected package keyboxd.
Preparing to unpack .../65-keyboxd_2.4.4-2ubuntu23_amd64.deb ...
Unpacking keyboxd (2.4.4-2ubuntu23) ...
Selecting previously unselected package gnupg.
Preparing to unpack .../66-gnupg_2.4.4-2ubuntu23_all.deb ...
Unpacking gnupg (2.4.4-2ubuntu23) ...
Selecting previously unselected package gnupg-l10n.
Preparing to unpack .../67-gnupg-l10n_2.4.4-2ubuntu23_all.deb ...
Unpacking gnupg-l10n (2.4.4-2ubuntu23) ...
Selecting previously unselected package gpg-wks-client.
Preparing to unpack .../68-gpg-wks-client_2.4.4-2ubuntu23_amd64.deb ...
Unpacking gpg-wks-client (2.4.4-2ubuntu23) ...
Selecting previously unselected package libalgorithm-diff-perl.
Preparing to unpack .../69-libalgorithm-diff-perl_1.201-1_all.deb ...
Unpacking libalgorithm-diff-perl (1.201-

Cloning into 'NuRaft'...


[None, None, None]


In [31]:

def compile_remote(i):

  


    command = 'gcloud compute ssh --zone "'+'us-central1-c'+'" "nuraft-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "\
    cd NuRaft; \
    ./prepare.sh; cp CMakeListsUbuntu.txt CMakeLists.txt;\
    mkdir build; cd build; cmake .. ; make -j4;"'

    print(command)

    output = os.system(command)
    print(output)



In [32]:
results = Parallel(n_jobs=60)(delayed(compile_remote)(i) for i in range(len(iplist)) )


print(results)  

+ . manifest.sh
++ ASIO_RELEASE=asio-1-24-0
++ pwd
+ PROJECT_DIR=/home/tejas/NuRaft
+ cd /home/tejas/NuRaft
+ '[' '!' -d asio/asio ']'
+ git clone https://github.com/chriskohlhoff/asio.git ./asio
Cloning into './asio'...
+ . manifest.sh
++ ASIO_RELEASE=asio-1-24-0
++ pwd
+ PROJECT_DIR=/home/tejas/NuRaft
+ cd /home/tejas/NuRaft
+ '[' '!' -d asio/asio ']'
+ git clone https://github.com/chriskohlhoff/asio.git ./asio
+ . manifest.sh
++ ASIO_RELEASE=asio-1-24-0
++ pwd
+ PROJECT_DIR=/home/tejas/NuRaft
+ cd /home/tejas/NuRaft
+ '[' '!' -d asio/asio ']'
+ git clone https://github.com/chriskohlhoff/asio.git ./asio
Cloning into './asio'...
Cloning into './asio'...
+ cd asio
+ git checkout asio-1-24-0
+ cd asio
+ git checkout asio-1-24-0
+ cd asio
+ git checkout asio-1-24-0
Note: switching to 'asio-1-24-0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by swit

-- The CXX compiler identification is GNU 14.2.0
-- Detecting CXX compiler ABI info
-- The CXX compiler identification is GNU 14.2.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- The CXX compiler identification is GNU 14.2.0
-- Detecting CXX compiler ABI info
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PkgConfig: /usr/bin/pkg-config (found version "1.8.1")
-- Checking for module 'libzstd'
--   Found libzstd, version 1.5.6
-- Detecting CXX compiler ABI info - done
-- C++ Standard: 11
-- Build type: RelWithDebInfo
-- Build Install Prefix : /usr/local
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found PkgConfig: /usr/bin/pkg-config (found version "1.8.1")
-- Checking for module 'libzstd'
--   Found libzstd, version 1.5.6
-- Detecting CXX compiler ABI info - done
-- Foun

/home/tejas/NuRaft/src/raft_server.cxx: In member function ‘void nuraft::raft_server::handle_peer_resp(nuraft::ptr<nuraft::resp_msg>&, nuraft::ptr<nuraft::rpc_exception>&)’:
/home/tejas/NuRaft/src/raft_server.cxx:840:28: warning: unused variable ‘rc’ [-Wunused-variable]
  840 |                 const auto rc = ctx_->cb_func_.call(cb_func::FollowerLost, &param);
      |                            ^~
/home/tejas/NuRaft/src/raft_server.cxx: In member function ‘bool nuraft::raft_server::check_leadership_validity()’:
/home/tejas/NuRaft/src/raft_server.cxx:1159:24: warning: unused variable ‘rc’ [-Wunused-variable]
 1159 |             const auto rc = ctx_->cb_func_.call(cb_func::FollowerLost, &param);
      |                        ^~
/home/tejas/NuRaft/src/raft_server.cxx: In member function ‘void nuraft::raft_server::handle_peer_resp(nuraft::ptr<nuraft::resp_msg>&, nuraft::ptr<nuraft::rpc_exception>&)’:
/home/tejas/NuRaft/src/raft_server.cxx:840:28: warning: unused variable ‘rc’ [-Wunused-va

[ 17%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/log_entry.cxx.o
[ 18%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/peer.cxx.o
[ 21%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/snapshot_sync_req.cxx.o
[ 19%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/raft_server.cxx.o
[ 21%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/snapshot_sync_req.cxx.o
[ 22%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/srv_config.cxx.o
[ 22%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/srv_config.cxx.o
[ 20%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/snapshot.cxx.o
[ 23%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/stat_mgr.cxx.o
[ 23%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/stat_mgr.cxx.o
[ 20%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/snapshot_sync_ctx.cxx.o


/home/tejas/NuRaft/src/raft_server.cxx: In member function ‘void nuraft::raft_server::handle_peer_resp(nuraft::ptr<nuraft::resp_msg>&, nuraft::ptr<nuraft::rpc_exception>&)’:
/home/tejas/NuRaft/src/raft_server.cxx:840:28: warning: unused variable ‘rc’ [-Wunused-variable]
  840 |                 const auto rc = ctx_->cb_func_.call(cb_func::FollowerLost, &param);
      |                            ^~
/home/tejas/NuRaft/src/raft_server.cxx: In member function ‘bool nuraft::raft_server::check_leadership_validity()’:
/home/tejas/NuRaft/src/raft_server.cxx:1159:24: warning: unused variable ‘rc’ [-Wunused-variable]
 1159 |             const auto rc = ctx_->cb_func_.call(cb_func::FollowerLost, &param);
      |                        ^~


[ 21%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/snapshot_sync_req.cxx.o
[ 22%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/srv_config.cxx.o
[ 23%] Building CXX object CMakeFiles/RAFT_CORE_OBJ.dir/src/stat_mgr.cxx.o
[ 23%] Built target RAFT_CORE_OBJ
[ 25%] Linking CXX shared library libnuraft.so
[ 25%] Linking CXX static library libnuraft.a
[ 23%] Built target RAFT_CORE_OBJ
[ 25%] Linking CXX static library libnuraft.a
[ 25%] Linking CXX shared library libnuraft.so
[ 25%] Built target static_lib
[ 28%] Building CXX object examples/CMakeFiles/stack_server.dir/stack_ops/stack_server.cxx.o
[ 28%] Building CXX object examples/CMakeFiles/set_server.dir/set_ops/set_server.cxx.o
[ 28%] Building CXX object examples/CMakeFiles/calc_server.dir/calculator/calc_server.cxx.o
[ 28%] Built target shared_lib
[ 29%] Building CXX object examples/CMakeFiles/stack_server.dir/logger.cc.o
[ 25%] Built target static_lib
[ 27%] Building CXX object examples/CMakeFiles/calc_server.dir/calcul

In file included from /home/tejas/NuRaft/examples/set_ops/set_server.cxx:18:
/home/tejas/NuRaft/examples/set_ops/set_state_machine.hxx: In member function ‘virtual nuraft::ptr<nuraft::buffer> set_state_machine::commit(ulong, nuraft::buffer&)’:
/home/tejas/NuRaft/examples/set_ops/set_state_machine.hxx:67:14: warning: unused variable ‘parse_result’ [-Wunused-variable]
   67 |         bool parse_result = static_cast<bool>(iss >> command);
      |              ^~~~~~~~~~~~
In file included from /home/tejas/NuRaft/examples/stack_ops/stack_server.cxx:18:
/home/tejas/NuRaft/examples/stack_ops/stack_state_machine.hxx: In member function ‘virtual nuraft::ptr<nuraft::buffer> stack_state_machine::commit(ulong, nuraft::buffer&)’:
/home/tejas/NuRaft/examples/stack_ops/stack_state_machine.hxx:68:14: warning: unused variable ‘parse_result’ [-Wunused-variable]
   68 |         bool parse_result = static_cast<bool>(iss >> command);
      |              ^~~~~~~~~~~~
In file included from /home/tejas/NuRa

[ 30%] Building CXX object examples/CMakeFiles/set_server.dir/logger.cc.o
[ 23%] Built target RAFT_CORE_OBJ
[ 25%] Linking CXX shared library libnuraft.so
[ 25%] Linking CXX static library libnuraft.a
[ 29%] Building CXX object examples/CMakeFiles/calc_server.dir/logger.cc.o
[ 25%] Built target static_lib
[ 27%] Building CXX object examples/CMakeFiles/set_server.dir/set_ops/set_server.cxx.o
[ 27%] Building CXX object examples/CMakeFiles/calc_server.dir/calculator/calc_server.cxx.o
[ 28%] Building CXX object examples/CMakeFiles/stack_server.dir/stack_ops/stack_server.cxx.o
[ 28%] Built target shared_lib
[ 29%] Building CXX object examples/CMakeFiles/set_server.dir/logger.cc.o
[ 31%] Building CXX object examples/CMakeFiles/stack_server.dir/in_memory_log_store.cxx.o
[ 32%] Building CXX object examples/CMakeFiles/set_server.dir/in_memory_log_store.cxx.o
[ 31%] Building CXX object examples/CMakeFiles/set_server.dir/logger.cc.o
[ 31%] Building CXX object examples/CMakeFiles/stack_server.dir/

In file included from /home/tejas/NuRaft/examples/set_ops/set_server.cxx:18:
/home/tejas/NuRaft/examples/set_ops/set_state_machine.hxx: In member function ‘virtual nuraft::ptr<nuraft::buffer> set_state_machine::commit(ulong, nuraft::buffer&)’:
/home/tejas/NuRaft/examples/set_ops/set_state_machine.hxx:67:14: warning: unused variable ‘parse_result’ [-Wunused-variable]
   67 |         bool parse_result = static_cast<bool>(iss >> command);
      |              ^~~~~~~~~~~~
In file included from /home/tejas/NuRaft/examples/stack_ops/stack_server.cxx:18:
/home/tejas/NuRaft/examples/stack_ops/stack_state_machine.hxx: In member function ‘virtual nuraft::ptr<nuraft::buffer> stack_state_machine::commit(ulong, nuraft::buffer&)’:
/home/tejas/NuRaft/examples/stack_ops/stack_state_machine.hxx:68:14: warning: unused variable ‘parse_result’ [-Wunused-variable]
   68 |         bool parse_result = static_cast<bool>(iss >> command);
      |              ^~~~~~~~~~~~


[ 32%] Building CXX object examples/CMakeFiles/set_server.dir/in_memory_log_store.cxx.o
[ 33%] Building CXX object examples/CMakeFiles/echo_server.dir/echo/echo_server.cxx.o
[ 33%] Linking CXX executable set_server
[ 34%] Linking CXX executable stack_server
[ 33%] Building CXX object examples/CMakeFiles/calc_server.dir/in_memory_log_store.cxx.o
[ 34%] Linking CXX executable stack_server
[ 34%] Built target set_server
[ 35%] Building CXX object examples/CMakeFiles/calc_server.dir/in_memory_log_store.cxx.o
[ 35%] Built target stack_server
[ 36%] Building CXX object examples/CMakeFiles/echo_server.dir/logger.cc.o
[ 35%] Building CXX object examples/CMakeFiles/echo_server.dir/echo/echo_server.cxx.o
[ 35%] Linking CXX executable set_server
[ 35%] Built target stack_server
[ 30%] Building CXX object examples/CMakeFiles/stack_server.dir/logger.cc.o
[ 36%] Building CXX object examples/CMakeFiles/quick_start.dir/quick_start.cxx.o
[ 37%] Building CXX object examples/CMakeFiles/quick_start.dir/qu

In [19]:
os.system('git add .; git commit -m "test"; git push -f origin HEAD:master')

[master 0be880e] test
 1 file changed, 67 insertions(+), 1 deletion(-)


To github.com:tejas-shivanand-mane/NuRaft.git
   fcc0232..0be880e  HEAD -> master


0

In [17]:

def git_pull(i):

  


    command = 'gcloud compute ssh --zone "'+'us-central1-c'+'" "nuraft-'+f"{i:03}"+'" --project "ucr-ursa-major-lesani-lab" --command "\
    cd NuRaft; \
    git pull;"'

    print(command)

    output = os.system(command)
    print(output)



In [20]:
results = Parallel(n_jobs=60)(delayed(git_pull)(i) for i in range(len(iplist)) )


print(results)  

From https://github.com/tejas-shivanand-mane/NuRaft
   fcc0232..0be880e  master     -> origin/master


Updating fcc0232..0be880e
Fast-forward
 .ipynb_checkpoints/gcp_run-checkpoint.ipynb |    6 +
 examples/example_custom.hxx                 |    4 +-
 gcp_run.ipynb                               | 2828 +++++++++++++++++++++++++++
 prepare_gcp.sh                              |   12 +
 4 files changed, 2848 insertions(+), 2 deletions(-)
 create mode 100644 .ipynb_checkpoints/gcp_run-checkpoint.ipynb
 create mode 100644 gcp_run.ipynb
 create mode 100644 prepare_gcp.sh


From https://github.com/tejas-shivanand-mane/NuRaft
   fcc0232..0be880e  master     -> origin/master


Updating fcc0232..0be880e
Fast-forward
 .ipynb_checkpoints/gcp_run-checkpoint.ipynb |    6 +
 examples/example_custom.hxx                 |    4 +-
 gcp_run.ipynb                               | 2828 +++++++++++++++++++++++++++
 prepare_gcp.sh                              |   12 +
 4 files changed, 2848 insertions(+), 2 deletions(-)
 create mode 100644 .ipynb_checkpoints/gcp_run-checkpoint.ipynb
 create mode 100644 gcp_run.ipynb
 create mode 100644 prepare_gcp.sh


From https://github.com/tejas-shivanand-mane/NuRaft
   fcc0232..0be880e  master     -> origin/master


Updating fcc0232..0be880e
Fast-forward
 .ipynb_checkpoints/gcp_run-checkpoint.ipynb |    6 +
 examples/example_custom.hxx                 |    4 +-
 gcp_run.ipynb                               | 2828 +++++++++++++++++++++++++++
 prepare_gcp.sh                              |   12 +
 4 files changed, 2848 insertions(+), 2 deletions(-)
 create mode 100644 .ipynb_checkpoints/gcp_run-checkpoint.ipynb
 create mode 100644 gcp_run.ipynb
 create mode 100644 prepare_gcp.sh
[None, None, None]
gcloud compute ssh --zone "us-central1-c" "nuraft-000" --project "ucr-ursa-major-lesani-lab" --command "    cd NuRaft;     git pull;"
0
gcloud compute ssh --zone "us-central1-c" "nuraft-001" --project "ucr-ursa-major-lesani-lab" --command "    cd NuRaft;     git pull;"
0
gcloud compute ssh --zone "us-central1-c" "nuraft-002" --project "ucr-ursa-major-lesani-lab" --command "    cd NuRaft;     git pull;"
0
gcloud compute ssh --zone "us-central1-c" "nuraft-000" --project "ucr-ursa-major-lesani-lab" --command " 

In [21]:

# os.system('gcloud compute instances delete --zone=us-central1-c --quiet $(gcloud compute instances list --format="value(name)")')


Deleted [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-000].
Deleted [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-001].
Deleted [https://www.googleapis.com/compute/v1/projects/ucr-ursa-major-lesani-lab/zones/us-central1-c/instances/nuraft-002].


0